In [13]:
import os
from dotenv import load_dotenv
import pandas as pd
from tqdm import tqdm
import torch
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb
from chromadb.config import Settings
from langchain_chroma import Chroma

In [2]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_huggingface import HuggingFacePipeline 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [ ]:
load_dotenv()

In [3]:
# Chroma 연결
CHROMA_HOST = os.getenv("CHROMA_HOST")  # 도커 네트워크에서 컨테이너명
CHROMA_PORT = int(os.getenv("CHROMA_PORT"))

client = chromadb.HttpClient(
    host=CHROMA_HOST,
    port=CHROMA_PORT,
    settings=Settings()
)

In [4]:
# 임베딩 모델 지정

model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'cuda'} 

embedding = HuggingFaceEmbeddings(
    model_name=model_name, 
    model_kwargs=model_kwargs, 
    encode_kwargs=model_kwargs,
    show_progress=True)

In [11]:
print(client.list_collections())

[Collection(name=financials), Collection(name=topics), Collection(name=news_articles)]


In [6]:
model_name = "google/gemma-3-12b-it" # "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",      
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
# news_articles, financials, topics collections들 가져오기(단순 로드)

news_articles_collection = Chroma(client=client, collection_name="news_articles", embedding_function=embedding)
financials_collection  = Chroma(client=client, collection_name="financials",   embedding_function=embedding)
topics_collection = Chroma(client=client, collection_name="topics",   embedding_function=embedding)

In [8]:
########### Inference setting code using RetrievalQAWithSourcesChain ############

topk_doc = 10

pipe = pipeline(
   "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=False,
    return_full_text=False,
    repetition_penalty=1.1,
    #no_repeat_ngram_size=3,
)

llm = HuggingFacePipeline(pipeline=pipe)


document_prompt = PromptTemplate(
    template="{page_content}",
    input_variables=["page_content"]
)

qa_prompt = PromptTemplate(
    template=(
        "아래 컨텐츠만을 근거로 질문에 한국어로 답하세요.\n\n"
        "모르면 모른다고 답하세요.\n\n"
        "질문에 언급된 이름들을 찾을 수 없을 때, 최대한 비슷한 맥락의 단어를 파악하여 질문에 충실하고 정확하게 답하세요.\n\n"
        "답변에 사용된 근거 컨텐츠를 원문 그대로 반드시 덧붙여서 답하세요.\n\n"
        "근거 컨텐츠를 덧붙일 때는 '출처: '와 같은 양식을 따르고, 출처는 컨텐츠 원문을 그대로 출력하세요.\n\n"
        "Question: {question}\n\n" # prompting한 question
        "Contents: {contents}\n\n" 
        "Anwser:" # 모델이 qeustions에 맞게 생성한 응답
    ),
    input_variables=["contents", "question"]
)

retriever=news_articles_collection.as_retriever(search_kwargs={"k": topk_doc}) 

###### 실행 순서 ######
# (1) as_retiever(): DB안에서 question에 맞는 topk 문서 추출(이때, Document 전체 반환)
# (2) document_prompt: as_retiever()에서 반환된 documents가 document_prompt에 전달되어 포맷팅(==contents)
# (3) qa_prompt: (2)에서 포맷팅된 값(==contents)이 qa_prompt를 거쳐 최종적으로 output을 generation

qa_chain = RetrievalQAWithSourcesChain.from_chain_type( 
    llm = llm,
    chain_type="stuff",
    return_source_documents=False,
    retriever=retriever,
    chain_type_kwargs={
        "document_prompt": document_prompt,
        "prompt": qa_prompt,
        "document_variable_name": "contents"
    }
)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [9]:
########### company별 keyword 추출: topics_collection ############

#추후 사용자의 질문에서 실제 회사명 키워드를 뽑아내는 작업이 필요함(자동화)

kw_query = '카카오' #컴퍼니 기준으로 키워드 찾기

keyword = kw_query
topk = 1

kw_topics = topics_collection.similarity_search(keyword, k=topk)

company=kw_topics[0].metadata.get('company')
com_to_key=kw_topics[0].metadata.get('keyword')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [10]:
print(kw_topics)

[Document(id='c54610b5-8095-4b19-8970-1ce8a8820be3', metadata={'company': '카카오', 'doc_type': 'topics', 'keyword': '자료처리'}, page_content='카카오 키워드: 자료처리')]

In [11]:
########### Output: question(company + keyword with company)에 따른 qa_chain에 결과 출력 코드 ############

# 자동화 시 코드 변경 필요
questions = f"기업 **{company}**의 기업 신용등급에 영향을 미칠만한 내용을 통해, **{com_to_key}** 업계의 동향을 파악하세요."

print(qa_chain({"question": questions}))

/tmp/ipykernel_230592/4265163844.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(qa_chain({"question": questions}))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True}. If this is not desired, please set these values explicitly.
/opt/conda/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=0), device(type='cuda', index=3), device(type='cuda', index=1), device(type='cuda', index=2)


{'question': '기업 **카카오**의 기업 신용등급에 영향을 미칠만한 내용을 통해, **자료처리** 업계의 동향을 파악하세요.', 'answer': "\n\n카카오의 기업 신용등급에 영향을 미치는 요소로는 다음과 같은 자료처리 업계 동향을 파악할 수 있습니다.\n\n*   **플랫폼 사업 성장 잠재력:** 카카오페이가 카카오톡과 카카오페이 앱을 활용한 플랫폼 접근성과 결제·금융 데이터를 모두 보유하고 있어 플랫폼 사업의 성장 잠재력이 크다는 평가가 있습니다.\n*   **맞춤형 서비스:** 신한, 하나, 롯데, BC, KB국민카드 등 다양한 카드사와 협력하여 사용자의 일상 속 결제 혜택에 초점을 맞춘 서비스를 제공하고 있습니다. 또한, 신용평가사 정보와 마이데이터, 카카오페이 결제 데이터를 활용해 사용자 맞춤형 카드 추천 서비스와 사전에 신용카드 발급 가능 여부와 한도를 확인해 볼 수 있는 서비스를 제공합니다.\n*   **소상공인 지원:** 카카오뱅크는 '소상공인 업종 특화 신용평가모형'을 통해 사업자 신용 평가의 변별력을 높이고, 자체 개발한 평가모형을 활용하여 뛰어난 사업역량을 보유한 개인사업자에게 대출 서비스를 제공합니다.\n*   **AI 기술 활용**: 카카오 AI 서비스 출시 등 IT 업계의 AI 전환이 본격화됨에 따라 Python, 머신러닝, 데이터 분석 등 AI 기초 역량을 학습하는 것이 중요합니다. 또한, KCA에서 수집한 전자파 측정 정보를 상용 앱에서도 확인할 수 있는 API를 개발하여 데이터 활용 능력을 보여주고 있습니다.\n*   **탄소 감축 노력:** 기보의 탄소 감축 기업 대상 지원과 마찬가지로, 카카오 역시 탄소 배출량 감소 노력을 통해 ESG 경영을 강화할 경우 긍정적인 영향을 받을 수 있습니다.\n\n출처:\n\n[ⓒ 카카오]카카오, 2분기 영업익 1859억…전년동기比 39%↑ '분기 최대 실적'[조윤정기자] 카카오는 연결 기준 올해 2분기 매출액이 전년 동기보다 1% 증가한 약 2조283억원을 기록했다고 7일 밝혔